<a href="https://colab.research.google.com/github/masterinfo/Machine_Learning_Pytorch_Colab/blob/main/Using_Optimizers_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using optimizers

In [61]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

Chargement des datas et normalisation

In [62]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

on ne se sers pas  de la classe

In [63]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    
    return x
    
#model = FMNIST()   

creation du model a 3 couches  784 entrés ( le spixels) =>128 neurones premmiere couche avec un un fonction d'activation Relu )=> 64 neurones sur la seconde couche avec une fonction d'activation Relu => couche de sortie de 10 neurones avec logsoftmax multiclasse

In [64]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [65]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [66]:
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)


premier passage juste pour comprendre l'effet de l'optimizer

In [67]:
output = model(images)
loss = criterion(output, labels)

print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

        

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  None


update the weights_grad ( calculs des gradients )

In [68]:
loss.backward()
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0030, -0.0030, -0.0030,  ..., -0.0030, -0.0030, -0.0030],
        [ 0.0022,  0.0022,  0.0022,  ...,  0.0024,  0.0022,  0.0022],
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        ...,
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [ 0.0021,  0.0021,  0.0021,  ...,  0.0022,  0.0021,  0.0021],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


Updat the weights ( neurones weights )

In [69]:
optimizer.step()

In [70]:
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[ 0.0012,  0.0206, -0.0279,  ...,  0.0234,  0.0052,  0.0035],
        [-0.0209, -0.0161, -0.0116,  ..., -0.0215, -0.0071, -0.0311],
        [-0.0202,  0.0148, -0.0334,  ..., -0.0204,  0.0011,  0.0079],
        ...,
        [ 0.0011, -0.0302,  0.0078,  ..., -0.0044,  0.0029,  0.0293],
        [-0.0243, -0.0231, -0.0075,  ...,  0.0105, -0.0334, -0.0168],
        [ 0.0290,  0.0047,  0.0107,  ...,  0.0267,  0.0331, -0.0124]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0030, -0.0030, -0.0030,  ..., -0.0030, -0.0030, -0.0030],
        [ 0.0022,  0.0022,  0.0022,  ...,  0.0024,  0.0022,  0.0022],
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        ...,
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [ 0.0021,  0.0021,  0.0021,  ...,  0.0022,  0.0021,  0.0021],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


Reset des gradients..( avant nouveau passage )

In [71]:
optimizer.zero_grad()

In [72]:
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[ 0.0012,  0.0206, -0.0279,  ...,  0.0234,  0.0052,  0.0035],
        [-0.0209, -0.0161, -0.0116,  ..., -0.0215, -0.0071, -0.0311],
        [-0.0202,  0.0148, -0.0334,  ..., -0.0204,  0.0011,  0.0079],
        ...,
        [ 0.0011, -0.0302,  0.0078,  ..., -0.0044,  0.0029,  0.0293],
        [-0.0243, -0.0231, -0.0075,  ...,  0.0105, -0.0334, -0.0168],
        [ 0.0290,  0.0047,  0.0107,  ...,  0.0267,  0.0331, -0.0124]],
       requires_grad=True)
Initial weights gradient :  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


application qui repars depuis le debut

In [73]:
model = FMNIST()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 3

for i in range(num_epochs):
    cum_loss = 0

    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()
       
     
    print(f"Training loss: {cum_loss/len(trainloader)}")

Training loss: 1.0100424232513412
Training loss: 0.5585998177909648
Training loss: 0.4902866960906271


**how to see loss for each batch of 1 epoch**

In [76]:
model = FMNIST()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 1

for i in range(num_epochs):
    cum_loss = 0
    batch_num=0

    for batch_num,(images, labels) in enumerate(trainloader,1):
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()
        print(f'Batch : {batch_num}, Loss : {loss.item()}')
     
    print(f"Training loss: {cum_loss/len(trainloader)}")

Batch : 1, Loss : 2.3215713500976562
Batch : 2, Loss : 2.3074803352355957
Batch : 3, Loss : 2.3164830207824707
Batch : 4, Loss : 2.297412157058716
Batch : 5, Loss : 2.2926530838012695
Batch : 6, Loss : 2.2953672409057617
Batch : 7, Loss : 2.3005030155181885
Batch : 8, Loss : 2.2910714149475098
Batch : 9, Loss : 2.2748677730560303
Batch : 10, Loss : 2.2863876819610596
Batch : 11, Loss : 2.268754720687866
Batch : 12, Loss : 2.268362522125244
Batch : 13, Loss : 2.286766529083252
Batch : 14, Loss : 2.2703473567962646
Batch : 15, Loss : 2.2913095951080322
Batch : 16, Loss : 2.2611446380615234
Batch : 17, Loss : 2.2757766246795654
Batch : 18, Loss : 2.27260160446167
Batch : 19, Loss : 2.27754807472229
Batch : 20, Loss : 2.273784875869751
Batch : 21, Loss : 2.230733633041382
Batch : 22, Loss : 2.2553842067718506
Batch : 23, Loss : 2.2328648567199707
Batch : 24, Loss : 2.255995035171509
Batch : 25, Loss : 2.2610247135162354
Batch : 26, Loss : 2.240332841873169
Batch : 27, Loss : 2.235386848449

nb d'images 60 000 , nb d'images par batch : 64 , nb de batvh  : 938

In [77]:
60000/64

937.5